# TD2 — Opérations OLAP (SQLite)

## Objectifs

- Requêtes OLAP : ROLLUP, DENSE_RANK, LAG, pivot.
- Contrôles cohérence.

In [ ]:
# Import SQLite (base TD1 chargée)
import sqlite3
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Charger données TD1 (simplifié)
# DDL et inserts omis pour brièveté, supposés exécutés
print('Base TD2 chargée')

## Requêtes OLAP

In [ ]:
# CA mensuel par produit et magasin (simplifié sans ROLLUP)
cursor.execute('''
SELECT strftime('%Y-%m-01', d.date_cal) AS mois, p.produit_nom, m.magasin_id, SUM(f.montant) AS ca_mensuel
FROM fact_ventes f
JOIN dim_date d ON f.date_id = d.date_id
JOIN dim_produit p ON f.produit_sk = p.produit_sk
JOIN dim_magasin m ON f.magasin_sk = m.magasin_sk
GROUP BY strftime('%Y-%m-01', d.date_cal), p.produit_nom, m.magasin_id
ORDER BY mois, ca_mensuel DESC;
''')
print('CA mensuel :')
for row in cursor.fetchall():
    print(row)

# Top 5 produits par région (simplifié)
cursor.execute('''
SELECT m.region, p.produit_nom, SUM(f.montant) AS ca
FROM fact_ventes f
JOIN dim_produit p ON f.produit_sk = p.produit_sk
JOIN dim_magasin m ON f.magasin_sk = m.magasin_sk
GROUP BY m.region, p.produit_nom
ORDER BY m.region, ca DESC
LIMIT 5;
''')
print('Top 5 produits :')
for row in cursor.fetchall():
    print(row)

# Variations MoM (simplifié)
cursor.execute('''
SELECT strftime('%Y-%m', d.date_cal) AS mois, p.categorie, SUM(f.montant) AS ca
FROM fact_ventes f
JOIN dim_date d ON f.date_id = d.date_id
JOIN dim_produit p ON f.produit_sk = p.produit_sk
GROUP BY strftime('%Y-%m', d.date_cal), p.categorie
ORDER BY mois, categorie;
''')
print('Variations (calcul manuel) :')
rows = cursor.fetchall()
for i, row in enumerate(rows):
    if i > 0 and row[1] == rows[i-1][1]:
        croissance = (row[2] - rows[i-1][2]) / rows[i-1][2] * 100 if rows[i-1][2] > 0 else 0
        print(f'{row[0]} {row[1]}: {croissance:.2f}%')

# Pivot simplifié
cursor.execute('''
SELECT p.produit_nom,
       SUM(CASE WHEN m.magasin_id = 'M01' THEN f.montant ELSE 0 END) AS ca_M01,
       SUM(CASE WHEN m.magasin_id = 'M02' THEN f.montant ELSE 0 END) AS ca_M02
FROM fact_ventes f
JOIN dim_produit p ON f.produit_sk = p.produit_sk
JOIN dim_magasin m ON f.magasin_sk = m.magasin_sk
GROUP BY p.produit_nom;
''')
print('Pivot CA :')
for row in cursor.fetchall():
    print(row)

## Contrôles cohérence

In [ ]:
# Total annuel
cursor.execute('''
SELECT strftime('%Y', d.date_cal) AS annee, SUM(f.montant) AS ca_annuel
FROM fact_ventes f
JOIN dim_date d ON f.date_id = d.date_id
GROUP BY strftime('%Y', d.date_cal);
''')
print('Total annuel :')
for row in cursor.fetchall():
    print(row)

# CA par magasin
cursor.execute('''
SELECT m.magasin_id, SUM(f.montant) AS ca_magasin
FROM fact_ventes f
JOIN dim_magasin m ON f.magasin_sk = m.magasin_sk
GROUP BY m.magasin_id;
''')
print('CA par magasin :')
for row in cursor.fetchall():
    print(row)

conn.close()
print('TD2 terminé')